# ***GERANDO A TABELA***

In [2]:
pip install geopy

In [3]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Carregamento dos arquivos
dfRide = pd.read_csv("ride_v2.csv", sep=";", dtype=str)
dfRideAdd = pd.read_csv("rideaddress_v1.csv", sep=";", dtype=str)
dfRideEst = pd.read_csv("rideestimative_v3.csv", sep=";", dtype=str)
dfProduct = pd.read_csv("product.csv", sep=";", dtype=str)

# 2. Uniformização: datas e RideID
dfRide["Schedule"] = pd.to_datetime(dfRide["Schedule"], errors="coerce")
for df in [dfRide, dfRideAdd, dfRideEst]:
    df["RideID"] = df["RideID"].astype(str).str.replace(".0", "", regex=False)

# 3. Derivar colunas de tempo
dfRide["Dia"] = dfRide["Schedule"].dt.weekday
dfRide["Hora"] = dfRide["Schedule"].dt.hour
dfRide["Minuto"] = dfRide["Schedule"].dt.minute
dfRide["HoraDecimal"] = dfRide["Hora"] + dfRide["Minuto"] / 60
dfRide["Faixa15min"] = dfRide["Schedule"].dt.floor("15min")
dfTempo = dfRide[["RideID", "Dia", "Hora", "Minuto", "HoraDecimal", "Faixa15min"]].dropna()

# 4. Extrair origem e destino (Lat, Lng, Address)
dfRideAdd = dfRideAdd.rename(columns={"RideAddressTypeID": "OrigDest"})
dfOrigem = dfRideAdd[dfRideAdd["OrigDest"] == "1"][["RideID", "Lat", "Lng", "Address"]].rename(
    columns={"Lat": "Lat1", "Lng": "Lng1", "Address": "AddressOrig"}
)
dfDestino = dfRideAdd[dfRideAdd["OrigDest"] == "2"][["RideID", "Lat", "Lng", "Address"]].rename(
    columns={"Lat": "Lat2", "Lng": "Lng2", "Address": "AddressDest"}
)
dfCoords = pd.merge(dfOrigem, dfDestino, on="RideID", how="inner")

# Corrige vírgulas e converte coordenadas
for col in ["Lat1", "Lng1", "Lat2", "Lng2"]:
    dfCoords[col] = dfCoords[col].str.replace(",", ".").astype(float).round(6)

# 5. Integrar todas as estimativas com produtos em UMA COLUNA tipo dicionário
dfRideEst["ProductID"] = dfRideEst["ProductID"].astype(str)
dfProduct["ProductID"] = dfProduct["ProductID"].astype(str)

dfEstimadaComProduto = pd.merge(dfRideEst, dfProduct, on="ProductID", how="left")
dfEstimadaComProduto["Price"] = dfEstimadaComProduto["Price"].str.replace(",", ".").astype(float)

dfEstimadaSelecionada = dfEstimadaComProduto.groupby("RideID").apply(
    lambda x: dict(zip(x["Description"], x["Price"]))
).reset_index().rename(columns={0: "Estimativas"})

# 6. Refiltra pelos RideID em comum
dfCoords["RideID"] = dfCoords["RideID"].astype(str)
dfEstimadaSelecionada["RideID"] = dfEstimadaSelecionada["RideID"].astype(str)
ids_comuns = set(dfTempo["RideID"]) & set(dfCoords["RideID"]) & set(dfEstimadaSelecionada["RideID"])

dfTempo = dfTempo[dfTempo["RideID"].isin(ids_comuns)].sort_values("RideID").reset_index(drop=True)
dfCoords = dfCoords[dfCoords["RideID"].isin(ids_comuns)].sort_values("RideID").reset_index(drop=True)
dfEstimadaSelecionada = dfEstimadaSelecionada[dfEstimadaSelecionada["RideID"].isin(ids_comuns)].sort_values("RideID").reset_index(drop=True)

# 7. Junta tudo sem merge (concatenando os DataFrames horizontalmente)
dfDerivado = pd.concat([
    dfTempo,
    dfCoords.drop(columns=["RideID"]),
    dfEstimadaSelecionada.drop(columns=["RideID"])
], axis=1)

# 8. Remove NaNs nas coordenadas
dfDerivado = dfDerivado.dropna(subset=["Lat1", "Lng1", "Lat2", "Lng2"]).reset_index(drop=True)

# 9. Cálculo da distância
dfDerivado["Distancia_km"] = dfDerivado.apply(
    lambda row: geodesic((row["Lat1"], row["Lng1"]), (row["Lat2"], row["Lng2"])).kilometers,
    axis=1
)

# Mostrar amostra do DataFrame final com estimativas e valor_uber
print("\n📊 Exemplo de dados do dfDerivado com estimativas:")
print(dfDerivado.head(10))

<ipython-input-3-3a1a1636f833>:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dfEstimadaSelecionada = dfEstimadaComProduto.groupby("RideID").apply(



📊 Exemplo de dados do dfDerivado com estimativas:
    RideID  Dia  Hora  Minuto  HoraDecimal          Faixa15min       Lat1  \
0  1183200    1    10       9    10.150000 2021-08-17 10:00:00 -26.329754   
1  1183201    1    10       9    10.150000 2021-08-17 10:00:00 -27.491979   
2  1183202    1    10      10    10.166667 2021-08-17 10:00:00 -19.849580   
3  1183203    1    10      10    10.166667 2021-08-17 10:00:00 -23.962423   
4  1183204    1    10      10    10.166667 2021-08-17 10:00:00 -10.919802   
5  1183205    1    10      10    10.166667 2021-08-17 10:00:00 -22.873502   
6  1183206    1    10      10    10.166667 2021-08-17 10:00:00 -23.554281   
7  1183207    1    10      10    10.166667 2021-08-17 10:00:00 -23.962423   
8  1183208    1    10      10    10.166667 2021-08-17 10:00:00 -19.849539   
9  1183209    1    10      10    10.166667 2021-08-17 10:00:00  -8.025771   

        Lng1                                        AddressOrig       Lat2  \
0 -48.840428  Rua João 

# ***TREINANDO O MODELO***

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# 🚗 Lista de serviços da Uber a prever
servicos_alvo = ["UberX", "Comfort", "Black"]

print("📈 Treinando modelos para serviços da Uber (com features auxiliares):\n")

# Listas para armazenar os resultados e os modelos treinados
resultados = []
modelos_treinados = {}  # <= AQUI salvaremos os modelos

for servico in servicos_alvo:
    print(f"🔮 Iniciando modelo para: {servico}")

    # Filtra somente onde o serviço alvo tem valor
    df_modelo = dfDerivado[dfDerivado["Estimativas"].apply(
        lambda d: isinstance(d, dict) and servico in d and isinstance(d[servico], (int, float))
    )].copy()

    if df_modelo.empty:
        print(f"⚠️ Nenhum dado encontrado para {servico}. Ignorado.\n")
        continue

    # Define a variável alvo
    df_modelo["y"] = df_modelo["Estimativas"].apply(lambda d: d[servico])

    # Features base
    features_base = ["Distancia_km", "Dia", "Hora", "HoraDecimal", "Lat1", "Lng1", "Lat2", "Lng2"]

    # Adiciona colunas auxiliares com os demais serviços (exceto o alvo)
    servicos_aux = set()
    df_modelo["Estimativas"].apply(lambda d: servicos_aux.update(d.keys()) if isinstance(d, dict) else None)
    servicos_aux.discard(servico)

    for s in servicos_aux:
        nome_coluna = f"aux_{s.lower().replace(' ', '_')}"
        df_modelo[nome_coluna] = df_modelo["Estimativas"].apply(lambda d: d.get(s) if isinstance(d, dict) else np.nan)

    # Prepara X e y
    features_auxiliares = [col for col in df_modelo.columns if col.startswith("aux_")]
    X = df_modelo[features_base + features_auxiliares].fillna(-1)
    y = df_modelo["y"]

    # Verificação mínima de dados
    if len(X) < 100:
        print(f"⚠️ Serviço '{servico}' com poucos dados após preparação ({len(X)} linhas) — ignorado.\n")
        continue

    # Treinamento
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    modelo = RandomForestRegressor(random_state=42)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # ✅ Salvando o modelo treinado
    modelos_treinados[servico] = modelo

    # Métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    resultados.append({
        "Serviço": servico,
        "Registros": len(X),
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "R²": round(r2, 4)
    })

    print(f"✅ Modelo treinado para: {servico}")
    print(f" → MAE  : R${mae:.2f}")
    print(f" → RMSE : R${rmse:.2f}")
    print(f" → R²   : {r2:.4f}\n")

# 🔚 Resumo final
if resultados:
    df_resultados = pd.DataFrame(resultados).sort_values(by="R²", ascending=False)
    print("📊 Resumo Final dos Modelos:")
    print(df_resultados.to_string(index=False))
else:
    print("❌ Nenhum modelo foi treinado.")

📈 Treinando modelos para serviços da Uber (com features auxiliares):

🔮 Iniciando modelo para: UberX
✅ Modelo treinado para: UberX
 → MAE  : R$0.48
 → RMSE : R$2.27
 → R²   : 0.9904

🔮 Iniciando modelo para: Comfort
✅ Modelo treinado para: Comfort
 → MAE  : R$2.20
 → RMSE : R$4.88
 → R²   : 0.9807

🔮 Iniciando modelo para: Black
✅ Modelo treinado para: Black
 → MAE  : R$1.82
 → RMSE : R$4.86
 → R²   : 0.9852

📊 Resumo Final dos Modelos:
Serviço  Registros  MAE  RMSE     R²
  UberX     235601 0.48  2.27 0.9904
  Black     123666 1.82  4.86 0.9852
Comfort     192876 2.20  4.88 0.9807


# ***INPUT CLIENTE***

In [17]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
from datetime import datetime

# Inicializa o geocodificador
geolocator = Nominatim(user_agent="uber-predict")

def endereco_para_coordenadas(endereco):
    try:
        local = geolocator.geocode(endereco)
        if local:
            return (local.latitude, local.longitude)
        else:
            return None
    except:
        return None

def calcular_distancia_km(coord1, coord2):
    return geodesic(coord1, coord2).km

def prever_por_endereco():
    print("📍 Previsão de Preço com Endereços")

    servico = input("👉 Escolha o serviço (UberX, Comfort, Black): ").strip()
    if servico not in modelos_treinados:
        print("❌ Modelo para esse serviço não foi treinado.")
        return

    origem = input("📌 Endereço de origem: ")
    destino = input("🎯 Endereço de destino: ")

    coord_origem = endereco_para_coordenadas(origem)
    coord_destino = endereco_para_coordenadas(destino)

    if not coord_origem or not coord_destino:
        print("❌ Não foi possível localizar um ou ambos os endereços.")
        return

    distancia_km = calcular_distancia_km(coord_origem, coord_destino)
    print(f"🛣️ Distância estimada: {distancia_km:.2f} km")

    # Hora e dia atuais
    agora = datetime.now()
    hora = agora.hour
    hora_decimal = agora.hour + agora.minute / 60
    dia_semana = agora.weekday()  # segunda = 0

    # Criar dados de entrada
    dados = {
        'Distancia_km': distancia_km,
        'Dia': dia_semana,
        'Hora': hora,
        'HoraDecimal': hora_decimal,
        'Lat1': coord_origem[0],
        'Lng1': coord_origem[1],
        'Lat2': coord_destino[0],
        'Lng2': coord_destino[1],
    }

    modelo = modelos_treinados[servico]
    for col in modelo.feature_names_in_:
        if col not in dados:
            dados[col] = -1  # preencher colunas auxiliares com -1

    df_input = pd.DataFrame([dados])
    preco = modelo.predict(df_input)[0]

    print(f"\n💰 Preço estimado para o serviço {servico}: R$ {preco:.2f}")

# ✅ Execute a função para testar
prever_por_endereco()

📍 Previsão de Preço com Endereços
👉 Escolha o serviço (UberX, Comfort, Black): Black
📌 Endereço de origem: Avenida Paulista, São Paulo
🎯 Endereço de destino: Parque Ibirapuera, São Paulo
🛣️ Distância estimada: 3.52 km

💰 Preço estimado para o serviço Black: R$ 22.06
